In [7]:
import numpy as np
from datasets import load_dataset
from torch.utils.data import DataLoader
import torch
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm
import matplotlib.pyplot as plt


BATCH_SIZE = 32
IMG_SIZE   = 256
NUM_CLASSES = 101
NUM_EPOCHS  = 5
LR          = 1e-4


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
dataset = load_dataset("food101")
#dataset = dataset.filter(lambda ex: ex["label"] < NUM_CLASSES)

train_ds = dataset["train"]
val_ds   = dataset["validation"]


preprocess = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
       mean=[0.485, 0.456, 0.406],   # ImageNet stats
       std =[0.229, 0.224, 0.225],
    ),
])

def preprocess_example(example):
    example['pixel_values'] = preprocess(example['image'])
    example['labels']       = example['label']
    return example

train_ds = train_ds.map(
    preprocess_example,
    remove_columns=['image','label'],
)
val_ds = val_ds.map(
    preprocess_example,
    remove_columns=['image','label'],
)

train_ds.set_format(type='torch', columns=['pixel_values','labels'])
val_ds.set_format(type='torch', columns=['pixel_values','labels'])

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False)

Map:   0%|          | 0/75750 [00:00<?, ? examples/s]

Map:   0%|          | 0/25250 [00:00<?, ? examples/s]

In [ ]:
model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

in_features = model.fc.in_features
model.fc = nn.Linear(in_features, NUM_CLASSES)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=LR)  # only fine-tuning fc

train_acc_list = []
val_acc_list   = []

for epoch in range(1, NUM_EPOCHS + 1):
    # Training
    model.train()
    correct_train = 0
    total_train   = 0
    for batch in tqdm(train_loader, desc=f"Train Epoch {epoch}"):
        imgs   = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss    = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(dim=1)
        correct_train += (preds == labels).sum().item()
        total_train   += labels.size(0)

    train_acc = correct_train / total_train
    train_acc_list.append(train_acc)

    # Validation
    model.eval()
    correct_val = 0
    total_val   = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Validate Epoch {epoch}"):
            imgs   = batch['pixel_values'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(imgs)
            preds   = outputs.argmax(dim=1)
            correct_val += (preds == labels).sum().item()
            total_val   += labels.size(0)

    val_acc = correct_val / total_val
    val_acc_list.append(val_acc)

    print(f"Epoch {epoch}/{NUM_EPOCHS} - Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

# ── 6) Plotting ──
epochs = np.arange(1, NUM_EPOCHS + 1)

plt.figure()
plt.plot(epochs, train_acc_list, label='Train Accuracy')
plt.plot(epochs, val_acc_list, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs. Validation Accuracy')
plt.legend()
plt.show()